# Распознавание рукописных цифр
В данной работе напишу нейронку на базе Pytorch CNN для распознавания собственных рукописных цифр. Сложность в том что, обученная сеть на MNIST на моих цифрах показывает низкие результаты. 

In [1]:
import os
print("Current working directory:", os.getcwd()) # current way
# directory = os.path.join(os.path.dirname(__file__), "my_folder")

Current working directory: c:\Python3\PyIntel\img_transform\Digits


In [2]:
import os
import json
from PIL import Image

import torch
import torch.utils.data as data
import torchvision.transforms.v2 as tfs
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.datasets import ImageFolder
# from torchvision import models

In [ ]:
if torch.xpu.is_available():
    device = 'xpu' # intel inside)

## Model
здесь в результате тестов подобрана максимально эффективная модель для моих рукописных цифр

In [ ]:
conv_model2 = nn.Sequential(
    nn.Conv2d(3, out_channels=16, kernel_size=3, stride=1, padding=1, bias=False),
    nn.BatchNorm2d(16),
    nn.ReLU(inplace=True),

    nn.Conv2d(16, out_channels=16, kernel_size=3, stride=1, padding=1, bias=False),
    nn.BatchNorm2d(16),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2),

    nn.Conv2d(16, out_channels=32, kernel_size=3, stride=1, padding=1, bias=False),
    nn.BatchNorm2d(32),
    nn.ReLU(inplace=True),

    nn.Conv2d(32, out_channels=20, kernel_size=3, stride=1, padding=1, bias=False),
    nn.BatchNorm2d(20),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2), # batch, out, 7, 7

    nn.Flatten(1), # batch, 980
    nn.Linear(20*7*7, 16, bias=False),
    nn.BatchNorm1d(16),
    nn.ReLU(inplace=True),

    nn.Linear(16, 16, bias=False),
    nn.BatchNorm1d(16),
    nn.ReLU(inplace=True),
    nn.Linear(16, 10)
)

In [ ]:
model = conv_model2
model.xpu()

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU(inplace=True)
  (10): Conv2d(32, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (11): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU(inplace=True)
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Flatten(start_dim=1, 

### формирование обучающих данных

In [ ]:
transforms = tfs.Compose([tfs.ToImage(),
                          tfs.ToDtype(torch.float32, scale=True),
                          ])

d_train = ImageFolder("dataset/train", transform=transforms)
train_data = data.DataLoader(d_train, batch_size=32, shuffle=True)

x_, y_ = next(iter(train_data))
x_[0].shape

torch.Size([3, 28, 28])

### train 
обучение и сохранение весов

In [ ]:
# optimizer = optim.Adam(params=model.parameters(), lr=0.01)
# loss_function = nn.CrossEntropyLoss()
# epochs = 8
# model.train()

# for _e in range(epochs):
#     loss_mean = 0
#     lm_count = 0

#     train_tqdm = tqdm(train_data, leave=True)
#     for x_train, y_train in train_tqdm:
#         x_train = x_train.xpu()
#         y_train = y_train.xpu()
#         predict = model(x_train)
#         loss = loss_function(predict, y_train)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         lm_count += 1
#         loss_mean = 1/lm_count * loss.item() + (1 - 1/lm_count) * loss_mean
#         train_tqdm.set_description(f"Epoch [{_e+1}/{epochs}], loss_mean={loss_mean:.3f}")

Epoch [8/8], loss_mean=0.028: 100%|██████████| 1875/1875 [00:38<00:00, 48.33it/s]


## Restore!

In [8]:
model.load_state_dict(torch.load('model_weight_9930.pth', map_location='xpu'))

<All keys matched successfully>

### test
результаты теста на обучающей выбрке

In [9]:
d_test = ImageFolder("dataset/test", transform=transforms)
test_data = data.DataLoader(d_test, batch_size=500, shuffle=False)

Q = 0

# тестирование обученной НС
model.eval()

for x_test, y_test in test_data:
    x_test = x_test.xpu()
    y_test = y_test.xpu()
    with torch.no_grad():
        p = model(x_test)
        p = torch.argmax(p, dim=1)
        Q += torch.sum(p == y_test).item()

Q /= len(d_test)
print(Q)

0.993


In [10]:
transform = tfs.Compose([
    tfs.ToImage(),
    tfs.Resize((28, 28)),                  # подгоняем под MNIST
    tfs.RandomInvert(p=1.0),               # инвертируем цвет
    tfs.ToDtype(torch.float32, scale=True)
])
# transform(img).shape

In [ ]:
# img = Image.open("images/im_2.png")
# img = img.convert("RGB")
# img = img.resize((28, 28))
# # tr = tfs.Compose([tfs.RandomInvert(p=1.0), tfs.Grayscale()])
# tr = tfs.Compose([tfs.RandomInvert(p=1.0)])
# img = tr(img)
# img = transform(img)
# img = img.to(device)
# torch.argmax(model(img.unsqueeze(0)).squeeze())

tensor(3, device='xpu:0')

### test
тест на моих цифрах

In [11]:
res_test = {}
with torch.no_grad():
    for i in range(10):
        img = Image.open(f"images/im_{i}.png")
        img = img.convert("RGB")
        tensor_img = transform(img)
        tensor_img = tensor_img.to(device)
        # res = model(tensor_img)
        res = model(tensor_img.unsqueeze(0)).squeeze()
        # res = res.softmax(dim=0) # .sort(descending=True)
        res = torch.argmax(res)
        res_test[i] = res.item()
        # res_test[i] = res
        
    
res_test

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}

В результате ряда экспериментов подобрана модель и приведен пример распознавания цифр не из обучающей выборки